create_search_grid.py -- James Sayre, contact: sayrejay@gmail.com

#### Purpose
- Written to create a search grid that will serve as units of analysis for the processing script (box_analysis.py), as well as the script that queries the Planet server for imagery (planet_api_querying.py). 
- This search grid displayed here is extremely simple. In practice, in our work we subset down to only search grids that are within the country extent, are not within cells of water or urban areas, and other criteria. I leave the subsetting of the search grid up to you, because the optimal grid to use will vary by the user.
- One very important choice to make is the projection of the grid. The script below assumes WGS84 (i.e. standard lat/lon coordinates), but if you are working with a relatively smaller area, I recommend use of the projection in which the Planet imagery are provided. This is to standardize the size of the images, for instance if you use a WGS84 projection for the grid boxes, then the resulting boxes will have varying length and width dimensions, and often appear to be rotated within the image. For our purposes (comparing images before/after with the same projection, and then computing a histogram), this is totally fine, as we don't care about the actual size of the input images. But if you are building a neural network that requires a consistent image size, this will be problematic and you will need to resize/crop the imagery. To give a sense of how large the projections are, for instance the Mexican state of Ensenada is covered by two separate projections -- EPSG: 32611 and EPSG: 32612. If you require a consistent image size, you will need to create separate grids for each projection.

### Outputs
- "search_regions_img_corr.csv" -- spreadsheet of the boxes/areas of interest (AOIs) that will need to be covered by satellite imagery

In [1]:
### Packages
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import regionmask

### Directories
if os.name == 'nt': ### Windows
    base_dir = "../../"
else: ### Non-windows
    base_dir = "../../"
intermediate_dir = os.path.join(base_dir,"Intermediates/")

### Outputs
search_region_satelite_imgs = os.path.join(intermediate_dir,"search_regions_img_corr.csv")

delta = 0.008333333 ### Spacing of grid 
xleft, xright = -117.395833, -86.6041679 ### Defining longitude range of AOI
ytop, ybottom = 32.79583333, 14.4041674 ### Defining latitude range of AOI
lons = np.arange(start=xleft, stop=xright + delta, step=delta)
lats = np.arange(start=ytop, stop=ybottom - delta, step=-delta)

lon_ind = dict(zip(lons, list(range(len(lons)))))
lat_ind = dict(zip(lats, list(range(len(lats)))))

Xgrid, Ygrid, = np.meshgrid(lons,lats)
grid_df = pd.DataFrame({'x':Xgrid.flatten(),'y':Ygrid.flatten()})

grid_df['i'] = grid_df['x'].apply(lambda x: lon_ind[x])
grid_df['j'] = grid_df['y'].apply(lambda x: lat_ind[x])
grid_df = grid_df.sort_values(['i','j']).reset_index(drop=True)

grid_df['image_id'] = ''
grid_df['bid'] = grid_df.index
grid_df['muncode'] = 2001 ### Code later on expects 5 digit unique region identifier

### Save to output file
grid_df.to_csv(search_region_satelite_imgs,index=False)